In [5]:
import seaborn as sns
import numpy as np
import pandas as pd
import pandas_gbq as pd_gbq
import gc
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from matplotlib import pyplot as plt
from google.cloud import bigquery
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score, roc_curve, auc
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.preprocessing import StandardScaler
import pickle
from tqdm.auto import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import joblib
import glob
import lightgbm as lgb
from google.cloud import storage
from pandas_profiling import ProfileReport
#from pyspark.context import SparkContext
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.regression import LinearRegression
#from pyspark.sql.session import SparkSession

%reload_ext google.cloud.bigquery


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage before optimization is: {:.2f} MB'.format(start_mem))
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


#Train data
bqclient = bigquery.Client()

query_string = """
SELECT * FROM `crack-petal-363512.Amex.Train`
"""

#select a.*,b.target from
#(SELECT * FROM `amexcds4.amextaiwan.amextaiwantrain`) a
#left join
#(SELECT * FROM `amexcds4.amextaiwan.trainLabels`) b
#on a.customer_ID	 = b.customer_ID;

dataframe = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)




In [8]:
#Train label
bqclient = bigquery.Client()

query_string = """
SELECT * FROM `crack-petal-363512.Amex.Train_Label`
"""

dataframe_labels = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)

In [9]:
# Reduce memory
df=reduce_mem_usage(dataframe)

Memory usage before optimization is: 8018.31 MB
Memory usage after optimization is: 2125.91 MB
Decreased by 73.5%


In [10]:
#del dataframe
del dataframe
gc.collect()

138

In [11]:
## Remove correlated columns > 87%
df.drop(['D_62'],axis=1,inplace=True) #D_77

df.drop(['D_103'],axis=1,inplace=True) #D_104
df.drop(['D_107'],axis=1,inplace=True)

df.drop(['D_139'],axis=1,inplace=True) #D_143
df.drop(['D_141'],axis=1,inplace=True)

df.drop(['B_1'],axis=1,inplace=True) #B_37
df.drop(['B_11'],axis=1,inplace=True)

df.drop(['B_7'],axis=1,inplace=True) #B_23

df.drop(['D_118'],axis=1,inplace=True)#D_119
df.drop(['D_115'],axis=1,inplace=True)

df.drop(['D_74'],axis=1,inplace=True)#D_75
df.drop(['D_58'],axis=1,inplace=True)

df.drop(['B_2'],axis=1,inplace=True)#B_33
df.drop(['B_18'],axis=1,inplace=True)

df.drop(['B_14'],axis=1,inplace=True)#B_15

df.drop(['B_16'],axis=1,inplace=True)#B_20

df.drop(['D_132'],axis=1,inplace=True)#D_131
df.drop(['D_79'],axis=1,inplace=True)


df.drop(['D_55'],axis=1,inplace=True)#D_48
df.drop(['D_61'],axis=1,inplace=True)

cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]

for cat_col in cat_features:
        encoder = LabelEncoder()
        df[cat_col] = encoder.fit_transform(df[cat_col])

In [12]:
def floorify_frac(x, T=1):
    """convert to int if float appears ordinal"""
    xt = (np.floor(x*T+1e-6)).fillna(-1)
    if np.max(xt)<=127:
        return xt.astype(np.int8)
    return xt.astype(np.int16) 

In [13]:
sampling_rate = {
 "B_4" : 78,
 "D_49" : 71,
 "D_51" : 3,
 "R_3" : 10,
 "R_2":1,
 "D_59" : 48,
 "D_65": 38,
 "D_59":48,
 "B_20" : 17,
 "B_22" : 2,
 "D_70" : 4,
 "D_72" : 3,
 "S_15" : 10,
 "D_75" : 15,
 "D_78": 2,
 "D_80":5,
 "R_13":31,
 "D_84" : 2,
 "R_16": 2,
 "R_17": 35,
 "R_18": 31,
 "D_89" : 9,
 "D_91" : 2,
 "D_111" : 2,
 "D_122": 7,
 "D_124": 22,
 "B_8": 1,
 "S_6" : 1,
 "R_4": 1,
 "R_10": 1,
 "D_81":1,
 "R_11": 2,
 "R_8":1,
 'D_83':1,
 'S_18':1,
 'D_86':1,
 'R_19':1,
 'B_32':1,
 'S_20':1,
 'R_20':1,
 'R_21':1,
 'B_33':1,
 'R_22':1,
 'R_23':1,
 'D_92':1,
 'D_93':1,
 'D_94':1,
 'R_24':1,
 'R_25':1,
 'D_96':1
    
}

for var, rate in sampling_rate.items():
    print("floorify", var)
    df[var] = floorify_frac(df[var],rate)

floorify B_4
floorify D_49
floorify D_51
floorify R_3
floorify R_2
floorify D_59
floorify D_65
floorify B_20
floorify B_22
floorify D_70
floorify D_72
floorify S_15
floorify D_75
floorify D_78
floorify D_80
floorify R_13
floorify D_84
floorify R_16
floorify R_17
floorify R_18
floorify D_89
floorify D_91
floorify D_111
floorify D_122
floorify D_124
floorify B_8
floorify S_6
floorify R_4
floorify R_10
floorify D_81
floorify R_11
floorify R_8
floorify D_83
floorify S_18
floorify D_86
floorify R_19
floorify B_32
floorify S_20
floorify R_20
floorify R_21
floorify B_33
floorify R_22
floorify R_23
floorify D_92
floorify D_93
floorify D_94
floorify R_24
floorify R_25
floorify D_96


In [14]:
## Remove Null values columns greater than 80% and for rest of null fill na ith median
print("Checking for null values\n")
null_col=[]
null_col_val=[]
null_col_less_than_80=[]
null_col_less_than_80_val=[]
null_col_less_than_50=[]
null_col_less_than_50_val=[]
null_col_less_than_10=[]
null_col_less_than_10_val=[]


for col in df.columns:
    tot_null = pd.isnull(df[col]).sum()
    pct_null = (float(tot_null) / df.shape[0]) * 100
    if pct_null >= 80:
       null_col.append(col)
       null_col_val.append(pct_null)
    if pct_null < 80 and pct_null >=50:
        null_col_less_than_80.append(col)
        null_col_less_than_80_val.append(pct_null)
    if pct_null < 50 and pct_null>10 :
        null_col_less_than_50.append(col)
        null_col_less_than_50_val.append(pct_null)    
    if pct_null < 10 and pct_null>0 :
        null_col_less_than_10.append(col)
        null_col_less_than_10_val.append(pct_null)   


for num_col in null_col_less_than_10:
    df[num_col].fillna((df[num_col].median()), inplace=True)

for num_col in null_col_less_than_50:
    df[num_col].fillna((df[num_col].median()), inplace=True)

for num_col in null_col_less_than_80:
    df[num_col].fillna((df[num_col].median()), inplace=True)

for num_col in null_col:
    df.drop(num_col,axis=1,inplace=True)

Checking for null values



In [15]:
# Change S_2 to date
df['S_2']=pd.to_datetime(df['S_2'])
df = df.sort_values(by=['customer_ID','S_2'])

In [16]:
# Scale data
#scaler = StandardScaler()
#df=scaler.fit_transform(df)

In [17]:
# Merge labels
df=df.merge(dataframe_labels, how='inner', on='customer_ID' )

In [18]:
# To save as parquet we need to convert float to 32
num_cols = list(df.dtypes[(df.dtypes == 'float32') | (df.dtypes == 'float16')].index)
for col in tqdm(num_cols):
    df[col] = df[col].astype(np.float32)

100%|██████████| 88/88 [00:02<00:00, 32.59it/s]


In [19]:
# Convert to parquet
table = pa.Table.from_pandas(df)
pqwriter = pq.ParquetWriter("train_bucketted.parquet", table.schema) 
pqwriter.write_table(table)
pqwriter.close()

### read parquet file

In [20]:
df=pd.read_parquet("train_bucketted.parquet")

In [21]:
df.drop('target',axis=1,inplace=True)

In [22]:
# Define categorical columns
cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
features = df.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
num_features = [col for col in features if col not in cat_features]

In [23]:
# Group by customer id numeric columns and do mean, std, min, max, last
df_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
df_num_agg.columns = ['_'.join(x) for x in df_num_agg.columns]
df_num_agg.reset_index(inplace = True)

In [24]:
# seems there will be some null due to std column as there 5120 rows where there is only one
#customer id row
df_num_agg.fillna(0, inplace=True)

In [25]:
# Group by customer id categorical columns and do count last and nunique
df_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
df_cat_agg.columns = ['_'.join(x) for x in df_cat_agg.columns]
df_cat_agg.reset_index(inplace = True)

In [26]:
# convery to float 32 numeric columns
cols = list(df_num_agg.dtypes[df_num_agg.dtypes == 'float64'].index)
for col in tqdm(cols):
    df_num_agg[col] = df_num_agg[col].astype(np.float32)

100%|██████████| 188/188 [00:13<00:00, 13.58it/s]


In [27]:
# convert to int32 categorical columns
cols = list(df_cat_agg.dtypes[df_cat_agg.dtypes == 'int64'].index)  
for col in tqdm(cols):
    df_cat_agg[col] = df_cat_agg[col].astype(np.int32)

100%|██████████| 33/33 [00:00<00:00, 71.94it/s]


In [28]:
# do difference of current row with the next row and pick the last row 
df1 = []
customer_ids = []
for customer_id, df2 in tqdm(df.groupby(['customer_ID'])):
    diff_df1 = df2[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
    df1.append(diff_df1)
    customer_ids.append(customer_id)

100%|██████████| 458913/458913 [08:20<00:00, 917.50it/s]


In [29]:
# add _diff to columns where difference was calculated in the earlier step
df1=np.concatenate(df1, axis = 0)
df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df2[num_features].columns])
df1['customer_ID'] = customer_ids

In [30]:
# seems there will be some null due to std column as there 5120 rows where there is only one
#customer id row
df1.fillna(0, inplace=True)

In [31]:
# Merge all numeric, categorical and difference dataframes
#df = df_num_agg.merge(df_cat_agg, how = 'inner', on = 'customer_ID').merge(df1, how = 'inner', on = 'customer_ID').merge(dataframe_labels, how = 'inner', on = 'customer_ID')
df = df_num_agg.merge(df_cat_agg, how = 'inner', on = 'customer_ID').merge(df1, how = 'inner', on = 'customer_ID')

In [32]:
# Round all numeroc columns
num_cols = list(df.dtypes[(df.dtypes == 'float32') | (df.dtypes == 'float64')].index)

In [33]:
for col in num_cols:
    df[col + '_round2'] = df[col].round(2) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [34]:
num_cols = [col for col in df.columns if 'last' in col]
num_cols = [col[:-5] for col in num_cols if 'round' not in col]

In [ ]:
# Create last minus mean difference column for all columns
for col in num_cols:
        try:
            df[f'{col}_last_mean_diff'] = df[f'{col}_last'] - df[f'{col}_mean']
        except:
            pass

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


In [ ]:
# Clean up all dataframes
del df_num_agg, df_cat_agg, df1
gc.collect()

43

In [ ]:
df=df.merge(dataframe_labels, how = 'inner', on = 'customer_ID')

## This is the place where we convert the cleaned up dataframe to CSV or Parquet format

In [ ]:
# Convert to CSV
#df.to_csv('train.csv')

In [ ]:
# Convert to Parquet
num_cols = list(df.dtypes[(df.dtypes == 'float32') | (df.dtypes == 'float16')].index)
for col in tqdm(num_cols):
    df[col] = df[col].astype(np.float32)
    
table = pa.Table.from_pandas(df)
pqwriter = pq.ParquetWriter("train_bucketted_grouped.parquet", table.schema) 
pqwriter.write_table(table)
pqwriter.close()

100%|██████████| 1494/1494 [00:01<00:00, 1193.75it/s]


In [ ]:
df.isna().sum().sum()

0